In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [24]:
# Параметри бази даних
DB_TYPE = "mysql"
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "yelp"
DB_USER = "root"
DB_PASSWORD = "PASSWORD"
TABLE_NAME = "yelp_attributes"

# Шлях до файлу CSV
CSV_FILE_PATH = "D:/Downloads/yelp_attributes.csv"

# Розмір одного чанк-оброблення (в рядках)
CHUNK_SIZE = 10000  # Оптимальний розмір для слабкого ПК

In [3]:
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [104]:
def clean_column_names(columns):
    """Очищення назв колонок для відповідності SQL."""
    return [col.replace('-', '_').replace(' ', '_').replace('.', '_') for col in columns]

def truncate_column(dataframe, column_name, max_length):
    """Обрізає значення в заданій колонці до максимальної довжини."""
    if column_name in dataframe.columns:
        dataframe[column_name] = dataframe[column_name].str.slice(0, max_length)
    return dataframe

def import_csv_to_sql_in_chunks(csv_path, table_name, engine, chunk_size=CHUNK_SIZE, max_length=500):
    total_rows = 0
    for chunk in pd.read_csv(
        csv_path, 
        chunksize=chunk_size, 
        low_memory=True,  # Зменшує використання пам'яті
        dtype=str  # Усі колонки читаються як текст, щоб уникнути помилок типу
    ):
        # Очищення заголовків колонок
        chunk.columns = clean_column_names(chunk.columns)

        # Обрізання значень у колонці `friends`
        # chunk = truncate_column(chunk, 'friends', max_length)
        chunk = truncate_column(chunk, 'date', max_length)

        # Запис у базу
        chunk.to_sql(table_name, engine, if_exists='append', index=False, chunksize=chunk_size)
        total_rows += len(chunk)
        print(f"Імпортовано {total_rows} рядків...")
    
    print(f"Імпорт завершено. Загалом імпортовано: {total_rows} рядків.")


In [105]:
# Виконання
try:
    import_csv_to_sql_in_chunks(CSV_FILE_PATH, TABLE_NAME, engine)
except Exception as e:
    print(f"Помилка: {e}")

Імпортовано 10000 рядків...
Імпортовано 20000 рядків...
Імпортовано 30000 рядків...
Імпортовано 40000 рядків...
Імпортовано 50000 рядків...
Імпортовано 60000 рядків...
Імпортовано 70000 рядків...
Імпортовано 80000 рядків...
Імпортовано 90000 рядків...
Імпортовано 100000 рядків...
Імпортовано 110000 рядків...
Імпортовано 120000 рядків...
Імпортовано 130000 рядків...
Імпортовано 140000 рядків...
Імпортовано 150000 рядків...
Імпортовано 150346 рядків...
Імпорт завершено. Загалом імпортовано: 150346 рядків.
